In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

%load_ext autoreload
%autoreload 2

import transformers
transformers.set_seed(17)


2024-06-11 09:50:55.146088: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# What should be the max length of the model?

Approach: Tokenize the longest annotation in the dataset, this is the max lenght.

In [ ]:
from src.dataset import TDMSDataset, PATH


# load all annotations:
dataset = TDMSDataset(PATH.TRAIN)

annotations = []
for i in range(len(dataset)):
    f, tex, jsn = dataset[i]
    annotations.append(str(jsn))


In [ ]:
import pandas as pd


df = pd.DataFrame(annotations)
df.columns = ["annotation"]
df

In [ ]:
df["str_len"] = df.apply(lambda row: len(row["annotation"]), axis=1)

# top 10 longest annotations by string lenght:
df_t10 = df.sort_values(by="str_len", ascending=False)[:10]
df_t10

In [ ]:
# ignoring the outlier of 56561 chars and all below 20k chars as candidates for longest annotation

df_longest = df_t10[1:6]
df_longest


In [ ]:
# is the longest annotation character wise also the longest annotation token wise?
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 9878 tokens => Max len should be 10k tokens

In [ ]:
# Sanity check: Other Tokenizers (llama)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8670 tokens => Max len should be 8700 tokens

In [ ]:
# Sanity check: Other Tokenizers (gemma)

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b")

df_longest["token_len"] = df_longest.apply(lambda row: len(tokenizer(row["annotation"])["input_ids"]), axis=1)
df_longest["token_len"].max()

# longest = 8898 tokens => Max len should be 9k tokens

# Model Experiments

In [ ]:
# Llama 3 8b vs Mistral 7b vs Gemma 7b

In [2]:
from src.models import Model


mistral_it = Model("mistralai/Mistral-7B-Instruct-v0.3", 0)
gemma_it = Model("google/gemma-7b-it", 1)
# llama_it = HFModel("meta-llama/Meta-Llama-3-8B-Instruct", 2)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
mistral_it.generate("Give an example of a kind of fish, reply be stating the name only.")

'\n\nTuna.'

In [7]:
test_text = """[!tp]\setlength{\tabcolsep}{0.5pt}
	\begin{center}
		\caption{Performance comparison on Oulu-CASIA database in terms of average classification accuracy of the 10-fold cross-validation when evaluating on three different test sets, including ``weak expression", ``peak expression" and ``combined", respectively.}
		\label{table:oulu_compare}
		\begin{tabular}{c|c|c|c}
			\hline\noalign{\smallskip}
			Method & weak expression & peak expression & combined\\
			\hline
			PPDN(standard SGD) &  67.05\% & 82.91\% &73.54\%\\	
			GoogLeNet (baseline) & 64.64\%& 79.21\% &71.32\%\\
			\hline
			PPDN  & \textbf{67.95\%}&\textbf{84.59\%} & \textbf{74.99\%}\\
			\hline
		\end{tabular}
	\end{center}"""

test_text2 = """Once the features were collected, they were shown to three separate individuals not present in the first experiment. They were shown the picture and asked to answer whether the given list of features found by the participants in the first experiment were present in the picture or not. Their responses are shown in the column labeled "Freq and Res" where a \tick represents that the corresponding participant believed the feature to be present in the picture. Not surprisingly, the features with the higher frequencies were answered correctly by all three users. On the other hand, some of the single frequency features were also answered correctly by all three users. The ones with indifferent answers are those that require a `keen' eye, e.g. ``hook''.
These experiments show that even a simple picture as the one shown in Figure 1 can have a lot of features, majority of which are very easy to answer but not so easy to extract. There can still be a lot more features present in the picture; one such example is "`one side of a rubik's cube"'.
This survey gives us some guidelines while choosing the pictures and/or secret questions:
\begin{itemize}
	\item Do not use pictures whose main object is the secret feature. So for example, if we chose the picture of Figure 1 as the challenge picture, then the secret question of "`Does the picture contain the digits 1-9?"' will certainly be a bad choice.
	\item Do not use simple pictures. Simple pictures contain very few features. This is evident from Figure 1. Although, one may still be able to think of more features, there does not seem to be a big number of features.
	\item Do not use secret questions which are hard to answer by the legitimate user. As an example, the feature "`Equilibrium"' in the above table was answered "`no"' by two users. This seems hard to find out in the picture and needs more of a philosophical eye. 
	\item Always allow for user error. So for example, if the user replies '10' pictures, allow an error of 2 to 3 wrong answers. This is clear from the picture that a user answered "`no"' to the feature "`Mathematics"', even though it seems to be describe the figure. 
\end{itemize}"""
mistral_it.generate(simple_fs(test_text2))

'[]'

In [3]:
from src.experiment_runner import Experiment, run
from src.dataset import PATH
from src.prompt_templates import simple_zs, simple_fs
from src.content_extraction import naive_doctaet, empty_to_unanswerable


exps = [
    # Experiment(mistral_it, simple_zs, naive_doctaet, empty_to_unanswerable),
    # Experiment(mistral_it, simple_fs, naive_doctaet, empty_to_unanswerable),
    Experiment(gemma_it, simple_zs, naive_doctaet, empty_to_unanswerable),
    Experiment(gemma_it, simple_fs, naive_doctaet, empty_to_unanswerable),
]

dfs = []
for exp in exps:
    # df = run(exp, PATH.VAL, max_iter=10)
    df = run(exp, PATH.VAL)
    dfs.append(df)

# for exp in exps:
#     df = run(exp, PATH.TEST)
#     dfs.append(df)

 94%|█████████████████████████████████████▌  | 94/100 [1:39:25<10:39, 106.55s/it]

: 

In [ ]:
run = "VAL_mistral-it_fs_initial-20240610-221117"

import pandas as pd


df = pd.read_feather(f"results/{run}/df.feather")

for annotation in df["annotation"][:10]:
    print(annotation)

In [9]:
from src.experiment_runner import free_cuda_memory

free_cuda_memory()